In [1]:
from fastai.vision.all import *
from tqdm import tqdm

In [2]:
df = pd.read_csv('data/train_ready.csv')
img_path = Path('data/train_features/')
mask_path = Path('data/train_labels/')

In [3]:
df.head(1)

,chip_id,location,datetime,cloudpath,is_valid,B02_path,B03_path,B04_path,B08_path,label_path
0,adwp,Chifunfu,2020-04-29T08:20:47Z,az://./train_features/adwp,False,data/train_features/adwp/B02.tif,data/train_features/adwp/B03.tif,data/train_features/adwp/B04.tif,data/train_features/adwp/B08.tif,data/train_labels/adwp.tif


In [4]:
IMG_SIZE = 512

In [5]:
def get_array(chip_folder, band):
    chip_folder = Path(chip_folder)
    fn = (chip_folder/band).with_suffix('.tif') ## Seems not to work with chip_folder from ColReader
    #fn = chip_folder + f'/{band}.tif'
    if Path(fn).exists():
        return (np.array(Image.open(fn)) / 27000)
    else:
        return np.zeros((512, 512))

def get_4chan(chip_folder):
    #ims = get_image_files(fold)
    #arrs = ims.map(Image.open).map(np.array)
    #stack = (np.stack(arrs, axis = -1) / 16000 * 255).astype(np.uint8)
    #assert(stack.shape[-1] == 4)
    blue  = get_array(chip_folder, 'B02')
    green = get_array(chip_folder, 'B03')
    red   = get_array(chip_folder, 'B04')
    #infra = get_array(chip_folder, 'B08')
    
    stack = np.stack([green, red, blue], axis = 0) ## 0
    
    return PILImage.create((stack*255).astype(np.uint8))# ## stack

---
## Datasets
---

In [6]:
def get_mask(chip_folder):
    fn = (mask_path / chip_folder.name).with_suffix('.tif')
    return tensor(np.array(Image.open(fn))).float()

In [9]:
def is_valid(chip):
    return df.loc[df['chip_id'] == chip.name, 'is_valid'].item()

In [10]:
chips = L(img_path.iterdir())
valid_chips = chips.argwhere(is_valid)
train_chips = chips.argwhere(is_valid, negate = True)

In [12]:
x_tfms = [get_4chan]
y_tfms = [get_mask]
splits = [train_chips, valid_chips]

dsets = Datasets(chips, [x_tfms, y_tfms], splits = splits)
dls = dsets.dataloaders(bs = 3)

In [13]:
b = dls.one_batch()

In [14]:
loss = CrossEntropyLossFlat(axis = 1)

In [19]:
x = b[0][0][None,...].to(device='cpu')
y = b[1][0][None,...].to(device='cpu')

In [20]:
y_hat = model(x)

In [21]:
self = loss
inp = y_hat.argmax(dim = self.axis)
targ = y

In [34]:
inp,targ = map(self._contiguous, (inp, targ))
print(inp.shape, targ.shape)

torch.Size([1, 512, 512]) torch.Size([1, 512, 512])


In [35]:
print(isinstance(inp, torch.Tensor))

True


In [36]:
print(self.floatify and targ.dtype != torch.float16)
print(targ.dtype in [torch.int8, torch.int16, torch.int32])

False
False


In [41]:
print(self.flatten and self.is_2d)
print(inp.view(-1, inp.shape[-1]).shape)
inp = inp.view(-1, inp.shape[-1])

True
torch.Size([512, 512])


In [50]:
R = y_hat.argmax(dim = self.axis).float().flatten()
S = y.flatten()
nn.CrossEntropyLoss(R,S)

RuntimeError: Boolean value of Tensor with more than one value is ambiguous

In [45]:
%debug

> /home/karrrrrrrr/miniconda3/envs/fastai/lib/python3.8/site-packages/torch/nn/functional.py(2824)cross_entropy()
   2822     if size_average is not None or reduce is not None:
   2823         reduction = _Reduction.legacy_get_string(size_average, reduce)
-> 2824     return torch._C._nn.cross_entropy_loss(input, target, weight, _Reduction.get_enum(reduction), ignore_index)
   2825 
   2826 

ipdb> input.type()
'torch.LongTensor'
ipdb> target.type()
'torch.FloatTensor'
ipdb> q


---
## End Datasets
---

In [ ]:
#class EmptyBlock(TransformBlock):
#    def __init__(self, type_tfms=None, item_tfms=None, batch_tfms=None, dl_type=None, dls_kwargs=None):
#        super().__init__(type_tfms=type_tfms, item_tfms=item_tfms, batch_tfms=batch_tfms, dl_type=dl_type, dls_kwargs=dls_kwargs)
#        self.item_tfms = L(item_tfms)
#        
#Simple4Chan = EmptyBlock(type_tfms = get_4chan)
#FourChanImageBlock = TransformBlock(type_tfms = get_4chan, batch_tfms = IntToFloatTensor)
#
#db = DataBlock(
#    blocks = (FourChanImageBlock, MaskBlock),
#    get_x = ColReader('chip_id', pref = img_path),
#    get_y = ColReader('label_path'),
#    splitter = ColSplitter(),
#    #item_tfms = Resize(IMG_SIZE),
#    #batch_tfms = setup_aug_tfms([#Rotate(), 
#    #                             RandomResizedCropGPU(IMG_SIZE, min_scale = 0.3), 
#    #                             #Brightness(0.3), 
#    #                             #Contrast(0.3), 
#    #                             #Hue(), 
#    #                             #Saturation(0.3)
#    #                           ])
#)
#
#dls = db.dataloaders(df, bs = 3)
#
#dls.show_batch()

In [17]:
img_size = dls.one_batch()[0].shape[-2:]

model = create_unet_model(
    arch = resnet34,
    n_out = 2,
    img_size = img_size,
)

_default_meta    = {'cut':None, 'split':default_split}
meta = model_meta.get(resnet34, _default_meta)

learn = Learner(
    dls,
    model,
    loss_func = CrossEntropyLossFlat(axis = 1),
    metrics = [Dice, JaccardCoeff],
    splitter = meta['split']
)

learn.freeze()

---

In [ ]:
#learn.lr_find()

In [ ]:
%debug

In [ ]:
#learn.fine_tune(5, 4e-5)

In [ ]:
#result_metric = None
#learn.export(f'res34_{IMG_SIZE}_j{result_metric}')